In [6]:
import torch
import gradientUtils as gu
# Define a threshold for early stopping
threshold = 1e-6
circle_center = torch.tensor([5.0, 5.0])  # Example circle center
radius = 3.0  # Example radius


In [7]:
def compute_closest_point_on_circle(x, y, circle_center, radius):
    # Calculate direction vector from point to circle center
    direction = torch.tensor([x - circle_center[0], y - circle_center[1]])
    # Normalize the direction
    norm_direction = direction / torch.norm(direction)
    # Scale by the circle's radius and add to circle center to get closest point
    closest_point = circle_center + norm_direction * radius
    return closest_point[0], closest_point[1]

In [8]:
def autograd(s_i, s_j, s_k, X_g, Y_g, destination):
    # Define an optimizer
    optimizer = torch.optim.Adam([s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y], lr=0.01)
    # Previous loss
    prev_loss = float("inf")
    # Training loop
    for epoch in range(100000):
        # Zero the gradients
        optimizer.zero_grad()

        # Compute the vertex
        X, Y = gu.compute_vertex(s_i, s_j, s_k)
        # Compute the vloss
        vloss = (X - X_g) ** 2 + (Y - Y_g) ** 2

        #Compute midpoint_loss
        midpoint_loss = gu.midpoint_loss(s_i, s_j, s_k, circle_center, radius)

        lambda_midpoint = 0.5  
        loss = vloss + lambda_midpoint * midpoint_loss
        # Early stopping condition
        if abs(prev_loss - loss.item()) < threshold:
            print(f"Stopping early at epoch {epoch} due to minimal loss change")
            gu.plot_and_save(
                epoch, s_i, s_j, s_k, X_g, Y_g, circle_center, radius, destination
            )
            break

        prev_loss = loss.item()
        # Backpropagate the error
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Print the loss
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            gu.plot_and_save(
                epoch, s_i, s_j, s_k, X_g, Y_g, circle_center, radius, destination
            )

    # Print the final site coordinates
    print(f"Final site coordinates:")
    print(f"s_i: ({s_i.x.item()}, {s_i.y.item()})")
    print(f"s_j: ({s_j.x.item()}, {s_j.y.item()})")
    print(f"s_k: ({s_k.x.item()}, {s_k.y.item()})")

    # Compute the final vertex
    final_X, final_Y = gu.compute_vertex(s_i, s_j, s_k)

    # Print the final vertex coordinates
    print(f"Final vertex coordinates: ({final_X.item()}, {final_Y.item()})")
    return s_i, s_j, s_k

In [9]:

# Define the ground truth vertex
X_g = torch.tensor([1.0], requires_grad=False)
Y_g = torch.tensor([1.0], requires_grad=False)
# Initialize the sites
s_i = gu.Site(1.0, 1.0)
s_j = gu.Site(2.0, 2.0)
s_k = gu.Site(1.50, 2.750)
destination = "images/autograd/bisec_optim/"

# Compute the vertex
X, Y = gu.compute_vertex(s_i, s_j, s_k)
# Compute the closest point on the circle to the initial vertex
X_g, Y_g = compute_closest_point_on_circle(X, Y , circle_center, radius)    

s_i, s_j, s_k = autograd(s_i, s_j, s_k, X_g, Y_g, destination)


Epoch 0, Loss: 159.79727172851562
Epoch 100, Loss: 18.277774810791016
Epoch 200, Loss: 2.9258933067321777
Epoch 300, Loss: 0.564609706401825
Epoch 400, Loss: 0.08953309804201126
Epoch 500, Loss: 0.013525612652301788
Epoch 600, Loss: 0.004581454209983349
Epoch 700, Loss: 0.003348571714013815
Epoch 800, Loss: 0.0027879623230546713
Epoch 900, Loss: 0.002337031066417694
Epoch 1000, Loss: 0.0019609364680945873
Epoch 1100, Loss: 0.0016505928942933679
Epoch 1200, Loss: 0.0013964292593300343
Epoch 1300, Loss: 0.0011884804116562009
Epoch 1400, Loss: 0.001017590519040823
Epoch 1500, Loss: 0.0008757929317653179
Epoch 1600, Loss: 0.0007563813123852015
Stopping early at epoch 1619 due to minimal loss change
Final site coordinates:
s_i: (2.514503240585327, 2.5178747177124023)
s_j: (3.234875440597534, 3.2489171028137207)
s_k: (1.9820616245269775, 3.1313464641571045)
Final vertex coordinates: (2.6130595207214355, 3.1412055492401123)


In [10]:
print("yhe")

yhe
